# Example 6 &mdash; Pytorch and Convolutional Neural Networks (CNNs)
## Introduction to CNNs
In a feedforward neural net, we can represent the weights of each layer in a matrix $W \in \mathbb{R}^{m \times n}$, where $n$ is the number of units below and $m$ is the number of units above. Given an activation vector $x \in \mathbb{R}^n$ of the layer below, we can compute the pre-activation vector of the layer above by $y = Wx$. Finally, we apply an activation function $\sigma$ to the pre-activations, which gives the new  activation vector $z = \sigma(y+b)$, where $b \in \mathbb{R}^m$ is the bias vector. 

<img src="files/neural_net2.jpeg" width=66%>

A CNN uses the exact same principles but with one important difference: **scalar elements are replaced by images**. Hence, the activation vector $x$ is now a vector of images instead of scalars. Dealing with images of shape $h \times w$, the vector $x$ now becomes a tensor $x \in \mathbb{R}^{n \times h \times w}$, but it is easier to think of it as a vector of images. In fact, denoting $\mathbb{F} = \mathbb{R}^{h \times w}$, we can write $x \in \mathbb{F}^n$, like in the feedforward case. 

The same change is done to the weights. Instead of scalar entries in our weight matrix, we now use small images, which are called **filters** or **kernels**. This turns the weight matrix into a rank-4 tensor $W \in \mathbb{R}^{n \times m \times k \times k}$, where $k$ determines the kernel size, which is usually square. Again, it is simpler to think in terms of matrices of images instead of tensors. We define $\mathbb{G} = \mathbb{R}^{k \times k}$ and write $W \in \mathbb{G}^{m \times n}$, like in the feedforward case. 

With this notation, the computation of the pre-activations works exactly as before, i.e. we compute a matrix-vector product $Wx$. The only difficulty is that we do not know how to multiply an element from $\mathbb{F}$ with an element from $\mathbb{G}$. This used to be a simple multiplication of two real numbers, but now we have to compose two images. This is where the convolution comes into play, that is we choose $f \ast g$ with $f \in \mathbb{F}, g \in \mathbb{G}$ as elementary operation in the matrix product. 

To fit to the usual definition of the 2D convolution, which is given by 
\begin{equation}
(f \ast g)[i, j] = \sum_{p=-\infty}^\infty \sum_{q=-\infty}^\infty f[p,q]g[i-p,j-q],
\end{equation}
we simply extend $f$ and $g$ by infinite zero-padding. The 2D convolution operation can be thought of as sliding the kernel $g$ over the input image $f$ as depicted in the following figure. 

<img src="files/depthcol.jpeg" width=50%>

Finally, the concepts of non-linearities and biases transfer trivially to CNNs. The non-linearity $\sigma$ is applied elementwise (i.e. pixelwise) to the pre-activation tensor. Biases do not change at all, i.e. we learn one bias weight per pre-activation image. 

These concepts suffice to build a first basic CNN like the one shown below. 

<img src="files/cnn.jpeg">

The figure shows the activations of a CNN. The dimension **depth** corresponds to $n$ in our notation. The resolution of the activation images can be changed by subsampling (called `stride` in pytorch). This can be done directly in the convolution operation or in a separate max-pooling operation, which reduces a $k \times k$-patch to a $1 \times 1$-patch by choosing its maximum value. 

Images were taken from http://cs231n.github.io/convolutional-networks/, which is a much more thorough introduction to the topic as this one. Also colah's blog provides intuitive explanations of the topics CNNs (http://colah.github.io/posts/2014-07-Conv-Nets-Modular/) and convolutions (http://colah.github.io/posts/2014-07-Understanding-Convolutions/).

## Task 1 &mdash; Run the Code
The code in this notebook implements a basic CNN in pytorch and trains it on MNIST, a standard dataset in machine learning. The task is to classify handwritten digits. Visit https://pytorch.org/ and follow the instructions to install pytorch on your platform and get the provided code (which is based on https://github.com/pytorch/examples/tree/master/mnist) up and running. After training for one epoch you should reach a test accurracy near 97%. 

## Task 2 &mdash; Make it Self-Normalizing
Self-Normalizing neural networks (SNNs, https://arxiv.org/abs/1706.02515) keep their activation distribution close to zero-mean and unti-variance throughout many layers. This is achieved by a special activation function, the scaled exponential linear unit (SELU). Although SNNs were designed as feedforward networks, we will just try it out on CNNs in this exercise. To make your network self-normalizing, we need to ensure three things:

1. Use `F.selu` as your activation function.
2. The weights of a SELU-activated layer must be initialized by a zero-mean normal distribution with a standard deviation of $1/\sqrt{n}$, where $n$ is the number of (scalar) weights used per SELU-activated unit. **Caution:** In feedforward nets $n$ is just the number of units in the layer below, but for CNNs it is slightly more complicated. 
3. If you use dropout, use SELU-dropout instead (does not apply in our case). 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from types import SimpleNamespace

In [2]:
# Hyperparameters
args = SimpleNamespace(batch_size=64, test_batch_size=1000, epochs=1,
                       lr=0.01, momentum=0.5, seed=1, log_interval=100)
torch.manual_seed(args.seed)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
# Data loader (downloads data automatically the first time)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [4]:
# CNN architecture (two conv layers followed by two fully connected layers)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
# This function trains the model for one epoch
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
# This function evaluates the model on the test data
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    

In [7]:
# Main
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438883
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.366815
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.327288
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.224575
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.336918
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.063451
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.117621
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.145470
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.117353

Test set: Average loss: 0.1016, Accuracy: 9662/10000 (97%)



We can see that Task 1 executes and the Accuracy is around 97% as to be expected. 

## Task 2

Initialising each activation layer with $\mathcal{N}(0, \frac{1}{\sqrt{n}})$

In [8]:
class Net_selu(nn.Module):
    def __init__(self):
        super(Net_selu, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        torch.nn.init.normal_(self.conv1, mean=0, std=1/np.sqrt(1*5*5))
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        torch.nn.init.normal_(self.conv2, mean=0, std=1/np.sqrt(20*5*5))
        self.fc1 = nn.Linear(4*4*50, 500)
        torch.nn.init.normal_(self.conv2, mean=0, std=1/np.sqrt(50*4*4))
        self.fc2 = nn.Linear(500, 10)
        # not necessary
        # torch.nn.init.normal_(self.conv2, mean=0, std=1/np.sqrt(500))


    def forward(self, x):
        x = F.selu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.selu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.selu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model_selu = Net().to(device)
optimizer_selu = optim.SGD(model_selu.parameters(), lr=args.lr, 
                      momentum=args.momentum)

for ep in range(1, args.epochs + 1):
    train(args, model_selu, device, train_loader, optimizer_selu, ep)
    test(args, model_selu, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318764
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.796757
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.493620
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.539716
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.255441
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.219816
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.049530
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.186546
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.242182
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.038131

Test set: Average loss: 0.1259, Accuracy: 9611/10000 (96%)

